Name: [Alper Ahmetoğlu](https://www.github.com/alper111)

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)
T = np.array(T).astype(np.float).T

Since $p(x_{1:N}|x_0) = p(x_1|x_0) p(x_2|x_1) \dots p(x_N|x_{N-1})$, given $x_0$ I can draw a sample from $p(x_1|x_0)$ and repeat the process for the succeeding $x$'s. $p(x_n | x_{n-1})$ corresponds to transition probabilities of letters because we are dealing with a Markov(1) model.

In [3]:
def sample_0(T, x_0, n):
    string = alphabet[x_0]
    x_t = x_0
    for i in range(n):
        x_t = np.random.multinomial(1, T[:,x_t]).argmax()
        string = string + alphabet[x_t]
    return string

In [4]:
for i in range(10):
    print(sample_0(T, np.random.randint(27), 6))

mulyorn
ey.mart
priza.w
freth.i
qus.pl.
plan..h
jut.ine
e.sinco
gse.fed
s.thach


In order to find $p(x_{-\alpha}|x_{\alpha})$, let's first try to find $p(x_t|x_{\alpha})$. This is to calculate a missing letter in some position given some other letters. Our aim is to decompose $p(x_{-\alpha}|x_{\alpha})$. For example, we can decompose $p(x_2, x_3 | x_1, x_4)$ as $p(x_2|x_1, x_4) p(x_3|x_2, x_4)$ since after we know $x_2$, $x_1$ is d-seperated from $x_3$. So with this strategy we can first sample a letter for some position, then repeat this process for all missing letters. Each missing letter is dependent on their first known left and right letter. For the query: "t_eb_ow___x", first missing letter is dependent on "t" and "e", second missing letter is dependent on "b" and "o" and other missing letters are dependent on "w" and "x".

In [5]:
def prob_t(T, p_xleft, p_xright, it_left, it_right):
    for i in range(it_left):
        p_xleft = np.matmul(T, p_xleft)
    
    for i in range(it_right):
        p_xright = np.matmul(p_xright, T)
    p_all = p_xleft * p_xright
    p_all = p_all / p_all.sum()
    return p_all

In [6]:
I = np.eye(27)

In [ ]:
prob_t(T, I[letter2idx['t']], I[])